# Generate topics using abstracts and keywords extracted from papers


Once keywords are extracted, this is an example on how to extract targeted topics (from these keywords) and associated different abstracts to the closest topics. In this example, I only focus in keywords of 3-grams 


In [1]:
import os 
import json
from collections import Counter


NEURIPS_ANALYSIS_DATA_PATH = '/var/data/neurips_analysis'

with open(os.path.join(NEURIPS_ANALYSIS_DATA_PATH, 'year_keywords_counter_n3.json'), 'r') as f:
    keywords_n3 = json.load(f)

print(f'keywords loaded -- 3-grams: {sum([len(v) for k, v in keywords_n3.items()])}')


all_n3_keywords = Counter()
for year, values in keywords_n3.items():
    all_n3_keywords.update([v[0] for v in values.items()])
selected_n3_keywords = [x[0] for x in all_n3_keywords.items() if x[1] > 1]
len(selected_n3_keywords)



keywords loaded -- 3-grams: 120341


2116

In [2]:
from collections import defaultdict

from tqdm.notebook import tqdm 

from papeles.paper.neurips import get_key
from papeles.utils import text as text_utils
from papeles.utils.topics import Topics

In [3]:
metadata_path = os.path.join(NEURIPS_ANALYSIS_DATA_PATH, 'files_metadata/')

metadata = {}
for filename in tqdm(os.listdir(metadata_path), 'loading metadata'):
    with open(os.path.join(metadata_path, filename), 'r') as f:  # open in readonly mode
        for line in f.readlines():
            data = json.loads(line)
            metadata[get_key(data['pdf_name'])] = data


## Topics extraction and topics per year


Topics are generated using the whole corpus (all abstracts from all years). Then, all topics are counter per year to generate a final analysis of which topics are the most common per year.  

In [4]:
topics = Topics([x['abstract'] for x in metadata.values()], selected_n3_keywords)
topics.topics

{'topic_0': ['high_dimensional_datasets',
  'low_dimensional_structures',
  'paper_concerns_problem',
  'problem_learning_control',
  'superposition-structured_dirty_statistical',
  'contextual_bandits_learner',
  'concerns_problem_learning',
  'low-rank_tensor_decomposition',
  'dynamic_assortment_selection',
  'posteriori_map_assignment'],
 'topic_1': ['empirical_risk_minimization',
  'hidden_markov_models',
  'tree_graphical_models',
  'gaussian_mixture_models',
  'interest_machine_learning',
  'variants_stochastic_gradient',
  'combinatorial_optimization_problems',
  'solve_wide_range',
  'sample_complexity_guarantees',
  'models_hidden_markov'],
 'topic_2': ['markov_decision_process',
  'decision_process_mdp',
  'paper_addresses_problem',
  'model_propose_algorithm',
  'unknown_markov_decision',
  'problem_learning_unknown',
  'problem_online_planning',
  'online_planning_markov',
  'planning_markov_decision',
  'state_action_spaces'],
 'topic_3': ['upper_lower_bounds',
  'general

In [5]:
topics_per_year = {}
for _, data in metadata.items():
    year = data['year']
    if year not in topics_per_year:
        topics_per_year[year] = Counter()
    
    topic_prediction = topics.predict_topics(data['abstract'])
    if sum(topic_prediction.values()) > 0:
        top_prediction = [x[0] for x in sorted(topic_prediction.items(), key=lambda x: x[1], reverse=True) if x[1] > 0][:5]
        topics_per_year[year].update(top_prediction)

topics_per_year

{2019: Counter({'topic_15': 1,
          'topic_29': 66,
          'topic_53': 11,
          'topic_47': 9,
          'topic_32': 105,
          'topic_2': 16,
          'topic_1': 14,
          'topic_51': 10,
          'topic_3': 13,
          'topic_8': 38,
          'topic_7': 123,
          'topic_25': 103,
          'topic_50': 64,
          'topic_52': 3,
          'topic_21': 36,
          'topic_14': 51,
          'topic_39': 11,
          'topic_18': 31,
          'topic_42': 7,
          'topic_35': 14,
          'topic_10': 8,
          'topic_9': 19,
          'topic_34': 46,
          'topic_46': 15,
          'topic_5': 39,
          'topic_36': 22,
          'topic_11': 29,
          'topic_20': 12,
          'topic_40': 36,
          'topic_37': 4,
          'topic_26': 16,
          'topic_33': 32,
          'topic_13': 5,
          'topic_23': 10,
          'topic_31': 12,
          'topic_38': 4,
          'topic_49': 11,
          'topic_24': 4,
          'topic_54

In [6]:
for year in range(2009, 2020):
    print(f'===============\n{year}')
    topics = topics_per_year[year]
    print(sorted(topics.items(), key=lambda x: x[1], reverse=True)[:5])
    

2009
[('topic_17', 9), ('topic_1', 6), ('topic_28', 6), ('topic_45', 6), ('topic_8', 5)]
2010
[('topic_5', 8), ('topic_7', 8), ('topic_32', 8), ('topic_1', 7), ('topic_18', 6)]
2011
[('topic_7', 7), ('topic_17', 7), ('topic_3', 6), ('topic_5', 6), ('topic_29', 6)]
2012
[('topic_23', 12), ('topic_5', 8), ('topic_35', 8), ('topic_13', 7), ('topic_3', 7)]
2013
[('topic_5', 22), ('topic_7', 15), ('topic_32', 13), ('topic_23', 11), ('topic_11', 11)]
2014
[('topic_7', 22), ('topic_5', 18), ('topic_25', 16), ('topic_32', 13), ('topic_29', 11)]
2015
[('topic_7', 28), ('topic_32', 28), ('topic_25', 25), ('topic_29', 21), ('topic_34', 14)]
2016
[('topic_7', 45), ('topic_32', 44), ('topic_25', 41), ('topic_29', 35), ('topic_50', 24)]
2017
[('topic_32', 59), ('topic_7', 58), ('topic_25', 44), ('topic_29', 39), ('topic_34', 28)]
2018
[('topic_7', 103), ('topic_32', 85), ('topic_25', 78), ('topic_29', 55), ('topic_50', 51)]
2019
[('topic_7', 123), ('topic_32', 105), ('topic_25', 103), ('topic_29', 6